# 모듈 임포트

In [ ]:
import time
import os
from IPython.display import clear_output

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers, Model, models, layers, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import Callback

from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications import EfficientNetB2


# 데이터 준비

In [ ]:
!wget https://github.com/dhrim/MDC_2021/raw/master/material/deep_learning/data/cat_and_dog.zip

In [ ]:
!unzip cat_and_dog.zip

In [ ]:
!apt-get update
!apt install tree

In [ ]:
!tree -d cat_and_dog/

In [ ]:
!ls -al ./cat_and_dog/train/Cat/*.jpg | wc
!ls -al ./cat_and_dog/train/Dog/*.jpg | wc
!ls -al ./cat_and_dog/test/Dog/*.jpg | wc
!ls -al ./cat_and_dog/test/Dog/*.jpg | wc

# CNN

In [ ]:
# copy from https://gist.github.com/stared/dfb4dfaf6d9a8501cd1cc8b8cb806d2e

# from IPython.display import clear_output
# from tensorflow.keras.callbacks import Callback

class PlotLosses(Callback):

  def on_train_begin(self, logs={}):
    self.epochs = []
    self.losses = []
    self.val_losses = []
    self.logs = []
    self.fig = plt.figure()


  def on_epoch_end(self, epoch, logs={}):

    self.epochs.append(epoch)
    self.losses.append(logs.get('loss'))
    self.val_losses.append(logs.get('val_loss'))

    clear_output(wait=True)
    plt.plot(self.epochs, self.losses, label="loss")
    plt.plot(self.epochs, self.val_losses, label="val_loss")
    plt.legend()
    plt.show();
    print("loss = ", self.losses[-1], ", val_loss = ", self.val_losses[-1])

In [ ]:
# import os

# import matplotlib.pyplot as plt
# import numpy as np

# from tensorflow.keras import optimizers
# from tensorflow.keras import models
# from tensorflow.keras import layers
# from tensorflow.keras import optimizers
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Dropout, Flatten, Dense, BatchNormalization
# from tensorflow.keras import Model
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.preprocessing.image import load_img
# from tensorflow.keras.preprocessing.image import img_to_array

# from tensorflow.keras.applications import efficientnet
# from tensorflow.keras.applications import EfficientNetB2

preprocessor = efficientnet.preprocess_input

conv_layers = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
conv_layers.trainable = False

model = models.Sequential()

model.add(conv_layers)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))   # <---

model.compile(loss='sparse_categorical_crossentropy', optimizer="RMSprop", metrics=['acc'])
model.summary()

BATCH_SIZE = 64

train_data_generator = ImageDataGenerator(
      rotation_range=10,
      width_shift_range=0.1,
      height_shift_range=0.1,
      horizontal_flip=True,
      vertical_flip=False,
      preprocessing_function=preprocessor
).flow_from_directory(
      "cat_and_dog/train",      # <---
      target_size=(224,224),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
)

test_data_generator = ImageDataGenerator(
      preprocessing_function=preprocessor
).flow_from_directory(
      "cat_and_dog/test",       # <---
      target_size=(224,224),
      batch_size=BATCH_SIZE,
      class_mode='sparse'
)

plot_losses = PlotLosses()      # <---
history = model.fit(
      train_data_generator,
      validation_data=test_data_generator,
      epochs=10,
      callbacks=[plot_losses]
)


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()


loss, acc = model.evaluate(test_data_generator)
print("loss=", loss)
print("acc=", acc)



test_x, test_y = test_data_generator.next()
y_ = model.predict(test_x)
predicted = np.argmax(y_, axis=-1)

plt.plot(test_y[:100], "o")
plt.plot(predicted[:100], '.')
plt.show()


custom_labels = list(test_data_generator.class_indices.keys())
print("label category index =", test_y[0])
print("predicted category index =", predicted[0])
print("predicted category name =", custom_labels[predicted[0]])


# 모델 저장

In [ ]:
model.save('my_model.h5')

In [ ]:
loaded_model = models.load_model('my_model.h5')

# 로드한 모델 평가

In [ ]:
loss, acc = loaded_model.evaluate(test_data_generator)
print('loss =', loss)
print('acc =', acc)

# 로드한 모델 예측

In [ ]:
y_ = loaded_model.predict(test_data_generator)
predicted = np.argmax(y_, axis=-1)
print(predicted)

# 이미지 그리면서 모델 예측

In [ ]:
batch_x, batch_y = next(test_data_generator)
print(batch_x.shape)
print(batch_y.shape)

In [ ]:
loaded_model.predict(batch_x)
y_ = loaded_model.predict(test_data_generator)
predicted = np.argmax(y_, axis=-1)
# print(predicted)

for i in range(5):
    plt.imshow(batch_x[i].astype(np.int))
    print('predicted={}, truth={}'.format(predicted[i], batch_y[i]))
    plt.show()

In [ ]:
plt.hist(batch_x.flatten(), bins=100)
plt.show()
print(batch_x[0].flatten())

# 다른 포맷으로 모델 저장

temsorflow 포맷

In [ ]:
model.save('my_model')

In [ ]:
loaded_model = models.load_model('my_model')

# weight 로 저장, 로드
모델의 가중치만 저장

In [ ]:
model.save_weights('my_model_weight')

In [ ]:
model = models.Sequential()

model.add(conv_layers)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='RMSprop', metrics=['acc'])
model.summary()

# model 가중치 로드
model.load_weights('my_model_weight')